# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [1]:
import matplotlib.pyplot as plt
import PIL.Image as PIL_image
import numpy as np

from importlib.machinery import SourceFileLoader
#In case of importing files from other directory
clip_pb2 = SourceFileLoader("clip_pb2","../protos/clip_pb2.py").load_module()
clip_pb2_grpc = SourceFileLoader("clip_pb2_grpc","../protos/clip_pb2_grpc.py").load_module()

import grpc
import io

## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data and call the service

In [37]:
from PIL import Image
import torch

#target='printart.isr.ist.utl.pt:8061'
target='localhost:8063'

texts = ["a diagram", "a dog", "a cat", "garden", "grass", "dirt road", "race car"]

file_paths = ["car.jpg","dog.jpg"]

image_byte_list = []
for path in file_paths:
    # Open the file in binary read mode ('rb') and read its entire content
    with open(path, 'rb') as f:
        image_bytes = f.read()
        image_byte_list.append(image_bytes)
        print(f"Read {path}: {len(image_bytes) / (1024 * 1024):.2f} MB")




request = clip_pb2.CLIPRequest(images= image_byte_list,
                               texts=texts) # list of strings

# List of file paths (example paths — replace with your actual ones)

channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = clip_pb2_grpc.CLIPServiceStub(channel)

Read car.jpg: 0.09 MB
Read dog.jpg: 0.37 MB


In [ ]:
#Name of the method in the service
response = estimator_stub.Forward(request)
#After finishing channel.close

I0000 00:00:1760612115.577101   58490 chttp2_transport.cc:1336] ipv6:%5B::1%5D:8063: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {http2_error:2, grpc_status:14}


In [24]:
response

image_emb: "PK\003\004\000\000\010\010\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\020\000\022\000archive/data.pklFB\016\000ZZZZZZZZZZZZZZ\200\002ctorch._utils\n_rebuild_tensor_v2\nq\000((X\007\000\000\000storageq\001ctorch\nFloatStorage\nq\002X\001\000\000\0000q\003X\003\000\000\000cpuq\004M\000\004tq\005QK\000K\002M\000\002\206q\006M\000\002K\001\206q\007\211ccollections\nOrderedDict\nq\010)Rq\ttq\nRq\013.PK\007\010\272o\222h\235\000\000\000\235\000\000\000PK\003\004\000\000\010\010\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\021\000$\000archive/byteorderFB \000ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZlittlePK\007\010\205=\343\031\006\000\000\000\006\000\000\000PK\003\004\000\000\010\010\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\016\000>\000archive/data/0FB:\000ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ\216\324\034>\360\023\210\276~\324\373\276\002l\214<\366k>>\276\357\264\276\207\375\030>\02

I0000 00:00:1760611835.577107   58490 chttp2_transport.cc:1336] ipv6:%5B::1%5D:8063: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {http2_error:2, grpc_status:14}


In [5]:
torch.load(io.BytesIO(response.similarity)).softmax(dim=-1)

tensor([[9.7670e-05, 1.8047e-04, 9.4234e-04, 1.3780e-05, 3.7842e-03, 2.6548e-03,
         9.9233e-01],
        [1.4549e-03, 3.1245e-01, 5.5151e-03, 8.3253e-03, 6.7070e-01, 1.9874e-04,
         1.3577e-03]])

In [10]:
torch.load(io.BytesIO(response.text_emb)).shape

torch.Size([7, 512])

In [14]:
torch.load(io.BytesIO(response.image_emb)).shape

torch.Size([2, 512])

I0000 00:00:1760610925.577172   58490 chttp2_transport.cc:1336] ipv6:%5B::1%5D:8063: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {grpc_status:14, http2_error:2}
